# Demo notebook : SelfPlay with ONNX models

## Setup

In [297]:
MODE = "local"  # "colab" | "colab-dev" | "local"

In [298]:
if MODE == "colab":
    !pip install -q lczerolens
elif MODE == "colab-dev":
    !rm -r lczerolens
    !git clone https://github.com/Xmaster6y/lczerolens -b main
    !pip install -q ./lczerolens

## Download ONNX Models

In [299]:
!gdown 1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO -O maia1100.onnx
!gdown 1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl -O maia1900.onnx

Downloading...
From: https://drive.google.com/uc?id=1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1100.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 37.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1900.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 30.4MB/s]


## Import Libraries

In [300]:
# ruff: noqa: E402
import importlib
import lczerolens.sampling as sampling
import lczerolens.search as search
import numpy as np

for module in (sampling, search):
    importlib.reload(module)

from lczerolens import LczeroBoard, LczeroModel
from lczerolens.sampling import MCTSSampler, ModelSampler, SelfPlay
from lczerolens.search import MaterialHeuristic, MCTS, Node

# 1. SelfPlay (MCTS + NN) Maia1100 VS Maia1900

## Load & Setup Model Samplers

In [301]:
maia_1100_model = LczeroModel.from_path("maia1100.onnx")
maia_1900_model = LczeroModel.from_path("maia1900.onnx")
white_sampler = MCTSSampler(model=maia_1100_model, num_simulations=100, use_argmax=True)
black_sampler = ModelSampler(model=maia_1900_model, use_argmax=True)

## Initialize SelfPlay

In [302]:
self_play = SelfPlay(white=white_sampler, black=black_sampler)
logs = []


def report_fn(log, to_play):
    logs.append((log, to_play))

## Play Game

In [303]:
board = LczeroBoard()

game_moves, final_board = self_play.play(board=board, max_moves=10, report_fn=report_fn)

## Display Results

In [304]:
print("Game moves:", game_moves)
print("Game result:", final_board.result())

Game moves: [Move.from_uci('e2e4'), Move.from_uci('c7c5'), Move.from_uci('g1f3'), Move.from_uci('d7d6'), Move.from_uci('f1c4'), Move.from_uci('e7e6'), Move.from_uci('b1c3'), Move.from_uci('g8f6'), Move.from_uci('d2d3'), Move.from_uci('f8e7')]
Game result: *


# 2. Test heuristic : Material advantage

In [305]:
fen = "6nr/pp3p1p/k1p5/8/1QN5/2P1P3/4KPqP/8 b - - 5 26"
iterations = 100
c_puct = 1.0
n_parallel_rollouts = 1

In [306]:
board = LczeroBoard(fen)
heuristic = MaterialHeuristic()
mcts = MCTS(c_puct=c_puct, n_parallel_rollouts=n_parallel_rollouts)
root = Node(board, parent=None)

In [307]:
mcts.search_(root, heuristic=heuristic, iterations=iterations)
best_move_idx = np.argmax(root.visits)
best_move = root.legal_moves[best_move_idx].uci()
print("Best move :", best_move)

Best move : g8e7


In [308]:
MCTS.render_tree(root, max_depth=30, save_to="./results/mcts_tree")

ImportError: graphviz is required to render trees, install it with `pip install lczerolens[viz]`.

# SelfPlay : Chose both heuristic to make play one against another

## Load & Setup Model Samplers

In [ ]:
# white_sampler = MaterialSampler(heuristic=MaterialHeuristic(), use_argmax=False)
# black_sampler = MaterialSampler(heuristic=MaterialHeuristic(), use_argmax=False)

## Initialize SelfPlay

In [ ]:
# self_play = SelfPlay(white=white_sampler, black=black_sampler)
# logs = []


# def report_fn(log, to_play):
#     logs.append((log, to_play))

## Play Game

In [ ]:
# board = LczeroBoard()

# game_moves, final_board = self_play.play(board=board, max_moves=100, report_fn=report_fn)

## Display Results

In [ ]:
# print("Game moves:", game_moves)
# print("Game result:", final_board.result())